In [3]:
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification
import numpy as np
from tqdm import tqdm

In [4]:
# Configuration
class Config:
    model_path = "custom_deberta_detector.pth"  # Your trained model
    base_model = "microsoft/deberta-base"       # Must match original training
    max_length = 256                            # Should match training setting
    batch_size = 8                              # Reduce if memory issues

# Initialize Tokenizer (fresh download)
tokenizer = DebertaTokenizer.from_pretrained(Config.base_model)

# Initialize Model Architecture
model = DebertaForSequenceClassification.from_pretrained(
    Config.base_model,
    num_labels=2  # Binary classification
)

# Load Trained Weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(Config.model_path, map_location=device))
model.to(device)
model.eval()
print(f"Model loaded on {device}")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on cpu


In [5]:
# Prediction Function
def predict(texts):
    """Batch prediction for multiple texts"""
    predictions = []
    for i in tqdm(range(0, len(texts), Config.batch_size), desc="Predicting"):
        batch = texts[i:i + Config.batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            max_length=Config.max_length,
            truncation=True,
            padding="max_length"
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            predictions.extend(batch_preds)
    
    return predictions

In [7]:
# Test Samples (Replace with your own)
test_samples = [
    "WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific research, infrastructure, public health and environmental protection. “The (Trump) administration has already been willing to say: ‘We’re going to increase non-defense discretionary spending ... by about 7 percent,’” Meadows, chairman of the small but influential House Freedom Caucus, said on the program. “Now, Democrats are saying that’s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I don’t see where the rationale is. ... Eventually you run out of other people’s money,” he said. Meadows was among Republicans who voted in late December for their party’s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. “It’s interesting to hear Mark talk about fiscal responsibility,” Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. “This is one of the least ... fiscally responsible bills we’ve ever seen passed in the history of the House of Representatives. I think we’re going to be paying for this for many, many years to come,” Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or “entitlement reform,” as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, “entitlement” programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryan’s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the “Dreamers,” people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. “We need to do DACA clean,” she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid.",
    "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images.",
    "Right now, the whole world is looking at the shocking fact that Democrat Doug Jones beat Republican Roy Moore in the special election to replace Attorney General Jeff Sessions in the United States Senate. Of course, Moore s candidacy was rocked by allegations of sexually harassing and even molesting teenage girls   and being banned from the mall in his hometown of Gadsden, Alabama for doing so.Even before that, Moore was an incendiary character in Alabama politics, having been removed as Chief Justice of the Alabama Supreme Court not once but twice, and having made statements such as Muslims should not be allowed in Congress and homosexuality should be illegal. Hell, he even said that the last time America was great was when we had slavery. Therefore, he was an extraordinarily damaged candidate as it was. However, despite all of this, Alabama is a deep red state, with many voters agreeing with some of Moore s more extreme positions, and some even insisting that the allegations of sexual misconduct were simply not true. That is why it was such a shock that Doug Jones pulled out a win for that Senate seat.Well, there is one entity that could not resist going all in on the fact that Roy Moore lost this race: Saturday Night Live. While doing a caricature of the results, SNL began, with its Weekend Update host Colin Jost brutally mocking Moore s alleged proclivities fore teen girls: Congratulations to Alabama s newest Senator   not Roy Moore. Doug Jones has become the first Democrat to win a Senate seat in Alabama in over 20 years. Said Roy Moore:  gross, over 20 years? Jost then got in a dig at Trump, for whom Moore s loss was  a humiliating failure, remind everyone what Trump said of Jones  win: The Republicans will have another shot at this seat in a very short period of time. It never ends! Indeed. If the sane people of America have anything to say about it, it will be a very, very long time after 2018 before the GOP is allowed to control anything. Jost continued mocking Trump: That s it? You just went all in for an accused pedophile and when he lost, [you re] just like,  well, we had fun! He could be removed from office tonight and tweet:  Congratulations to Robert Mueller on a great investigation. Had a fun time being president. Catch you on the flippity-flop! #DietCokeTime . Oh, if only that could be our reality, to have Trump congratulating Mueller for removing him and his entire treasonous, criminal administration. Until then, we ll have to stick to  Weekend Update  and the rest of SNL, and hope for the best.Watch the video below:Featured image via Scott Olson/Getty Images",
    "WASHINGTON (Reuters) - U.S. President Donald Trump signed Republicans’ massive $1.5 trillion tax overhaul into law on Friday, cementing the biggest legislative victory of his first year in office, and also approved a short-term spending bill that averts a possible government shutdown. Trump said he wanted to sign the tax bill before leaving Washington on Friday for his Mar-a-Lago estate in Florida, rather than stage a more formal ceremony in January, so he could keep his promise to finish work before Christmas. “I didn’t want you folks to say I wasn’t keeping my promise. I’m keeping my promise,” he told reporters in the White House. The two pieces of legislation represent Trump’s most significant accomplishment with Congress since taking office in January, as well as a sign of what awaits when he returns from Florida after the Christmas holiday. The tax package, the largest such overhaul since the 1980s, slashes the corporate rate from 35 percent to 21 percent and temporarily reduces the tax burden for most individuals as well. Trump praised several companies that have announced employee bonuses in the wake of the bill’s passage, naming AT&T, Boeing, Wells Fargo, Comcast  and Sinclair Broadcast Group. “Corporations are literally going wild over this,” he said.      Democrats had opposed the bill as a giveaway to the wealthy that would add $1.5 trillion to the $20 trillion national debt during the next decade. The spending bill extends federal funding through Jan. 19, largely at current levels. It does nothing to resolve broader disputes over immigration, healthcare and military spending. Republicans also are divided over whether to follow up their sweeping overhaul of the U.S. tax code with a dramatic restructuring of federal benefit programs. House Speaker Paul Ryan has said he would like to revamp welfare and health programs but Senate Republican Leader Mitch McConnell told National Public Radio on Monday that he was not interested in cutting those programs without Democratic support. Trump’s year also closes with significant turnover of many top staffers who had been in the White House since early in his term. On Friday, the White House confirmed Deputy Chief of Staff Rick Dearborn and Jeremy Katz, who worked under White House economic adviser Gary Cohn, were leaving.",
    "(Reuters) Donald Trump, while announcing the tariffs said, “Their Prime Minister [Narendra Modi] just left [US recently]. He is a great friend of mine, but I said to him that 'you're a friend of mine, but you've not been treating us right'. India charges us 52 per cent, so we will charge them half of that - 26 per cent.",
    "India and the United States have purportedly finalised the terms of reference for talks on the first segment of a bilateral trade agreement and were ahead of other countries in negotiating with America, reported Reuters, citing an anonymous official."
]

# Get predictions (0=Real, 1=Fake)
print("\nTesting model on sample texts...")
preds = predict(test_samples)

# Display results
print("\nPredictions:")
for text, pred in zip(test_samples, preds):
    print(f"\nText: {text[:80]}..." if len(text) > 80 else f"Text: {text}")
    print(f"Prediction: {'FAKE' if pred == 1 else 'REAL'}")

# Interactive Testing
def interactive_test():
    """Test custom inputs"""
    print("\nInteractive mode (type 'quit' to exit)")
    while True:
        text = input("\nEnter text to analyze: ")
        if text.lower() in ['quit', 'exit']:
            break
        pred = predict([text])[0]
        print(f"Prediction: {'FAKE' if pred == 1 else 'REAL'}")

# Uncomment to enable interactive testing
# interactive_test()


Testing model on sample texts...


Predicting: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s]


Predictions:

Text: WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S....
Prediction: REAL

Text: Donald Trump just couldn t wish all Americans a Happy New Year and leave it at t...
Prediction: FAKE

Text: Right now, the whole world is looking at the shocking fact that Democrat Doug Jo...
Prediction: FAKE

Text: WASHINGTON (Reuters) - U.S. President Donald Trump signed Republicans’ massive $...
Prediction: REAL

Text: (Reuters) Donald Trump, while announcing the tariffs said, “Their Prime Minister...
Prediction: REAL

Text: India and the United States have purportedly finalised the terms of reference fo...
Prediction: FAKE
